#**Sistemi Digitali M 2020/21 - UNIBO** 
##by Pier Domenico d'Amicis

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


For the code cell below you should see ‘Found GPU’ and tf version 1.x.
Remember to change runtime to GPU

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Found GPU at: /device:GPU:0
1.15.2


Clone the Tensorflow API repo

In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 1211, done.
remote: Counting objects: 100% (1211/1211), done.
remote: Compressing objects: 100% (675/675), done.
remote: Total 55883 (delta 731), reused 957 (delta 522), pack-reused 54672
Receiving objects: 100% (55883/55883), 572.23 MiB | 15.82 MiB/s, done.
Resolving deltas: 100% (38340/38340), done.


Install some needed tools and dependencies.

In [ ]:
%cd "/content/models/research"
!mv object_detection/ /content/
!mv slim/ /content/
%cd "/content"
%rm -rf models

/content/models/research
/content


In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 31 not upgraded.
Need to get 1,614 kB of archives.
After this operation, 8,907 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd

In [ ]:
%cd /content/
!protoc object_detection/protos/*.proto --python_out=.

/content


In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/:/content/slim'
%cd /content/slim

/content/slim


Always run the cell below for every session restart

In [ ]:
!python setup.py build
!python setup.py install

running build
running build_py
creating build
creating build/lib
creating build/lib/deployment
copying deployment/model_deploy_test.py -> build/lib/deployment
copying deployment/__init__.py -> build/lib/deployment
copying deployment/model_deploy.py -> build/lib/deployment
creating build/lib/nets
copying nets/post_training_quantization.py -> build/lib/nets
copying nets/inception_v4_test.py -> build/lib/nets
copying nets/overfeat.py -> build/lib/nets
copying nets/mobilenet_v1_eval.py -> build/lib/nets
copying nets/mobilenet_v1_test.py -> build/lib/nets
copying nets/inception_v1.py -> build/lib/nets
copying nets/dcgan.py -> build/lib/nets
copying nets/resnet_v2.py -> build/lib/nets
copying nets/mobilenet_v1.py -> build/lib/nets
copying nets/inception.py -> build/lib/nets
copying nets/inception_v2.py -> build/lib/nets
copying nets/i3d.py -> build/lib/nets
copying nets/__init__.py -> build/lib/nets
copying nets/resnet_v2_test.py -> build/lib/nets
copying nets/inception_v4.py -> build/lib/ne

Run Code below to check if all we need for the training has been installed.

In [ ]:
%cd /content/object_detection/builders/
!python ./model_builder_tf1_test.py

/content/object_detection/builders
Running tests under Python 3.7.10: /usr/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[ RUN      ] ModelBuilderTF1Test.test_cre

Clone my own repo to get

*   a python script to converter dataset FROM CSV FORMAT TO TFRECORD FORMAT
*   labels.pbtxt
*   network.config (mobilnet_ssd_v1 modified config file) 


**Warning** you need to modify *labels.pbtxt* and *network.config*(optionally) according to your dataset.

In [ ]:
%cd /content/object_detection/
%mkdir training
%rm -rf object_detection_ssd
!git clone https://github.com/P-damicis/object_detection_ssd.git
!mv object_detection_ssd/generate_tfrecord.py ./
!mv object_detection_ssd/network.config training/
!mv object_detection_ssd/labels.pbtxt training/
%rm -rf object_detection_ssd

/content/object_detection
Cloning into 'Sistemi_Digitali_M'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 36 (delta 13), reused 16 (delta 3), pack-reused 0
Unpacking objects: 100% (36/36), done.


Import your own dataset from roboflow.com in CSV format.

*** If you import it in TFRECORD format it will not necessary run generate_tfrecord.py

In [ ]:
%cd /content/object_detection
%mkdir Images
%cd Images
!curl -L "YOUR DOWNLOAD LINK FROM ROBOFLOW.COM" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
%rm -rf valid
!mv train/_annotations.csv ../data/train_labels.csv
!mv test/_annotations.csv ../data/test_labels.csv


###Conversion to obtain **tfrecord**

In [ ]:
%cd /content/object_detection/

# Create train data:
!python generate_tfrecord.py \
--label='YOUR_CLASS_NAME' \
--csv_input=data/train_labels.csv \
--img_path=Images/train \
--output_path=data/train.record

# Create test data:
!python generate_tfrecord.py \
--label='YOUR_CLASS_NAME' \
--csv_input=data/test_labels.csv \
--img_path=Images/test \
--output_path=data/test.record


Note: train.py is in legacy folder

#**TRAINING**

In [ ]:
%cd /content/object_detection/legacy/
!python train.py \
--logtostderr \
--train_dir=../training/checkpoints/ \
--pipeline_config_path=../training/network.config

##Exporting inference graph from trained checkpoint 

In [ ]:
%cd /content/object_detection/training/checkpoints/
%ls

/content/object_detection/training/checkpoints
checkpoint                                   model.ckpt-0.index
events.out.tfevents.1618300831.f28faaecdca4  model.ckpt-0.meta
graph.pbtxt                                  pipeline.config
model.ckpt-0.data-00000-of-00001


Save inference graph


In [ ]:
%cd /content/object_detection/

!python export_inference_graph.py \
--input_type image_tensor \
--pipeline_config_path training/network.config \
--trained_checkpoint_prefix training/checkpoints/model.ckpt-15000 \
--output_directory trained_inference_graph/

##Preparing to TFLite Conversion

In [ ]:
import tensorflow as tf

In [ ]:
%cd /content/object_detection

!python export_tflite_ssd_graph.py \
--pipeline_config_path training/network.config \
--trained_checkpoint_prefix trained_inference_graph/model.ckpt \
--output_directory /content/tflite \
--add_postprocessing_op false \
--max_detections 1 \


#**Conversion to TFlite Model**

In [ ]:
!pip install tensorflow==2.1.0
!pip install tf-nightly
import tensorflow as tf

if we want to quantize int8 we have to create the function *representative_dataset* that is required by the converter.

This function provides a sample of images taken from the train set

In [ ]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt

def representative_dataset():
  imagespath = "/content/object_detection/Images/train/"
  for file in os.listdir(imagespath):
    data = plt.imread(imagespath+file)
    data = np.expand_dims(data, axis=0)
    yield [data.astype(np.float32)]

tf.__version__

Creating TFLite converter to convert our frozen graph.
You need to know the name of input and output node of the graph.

In [ ]:
import tensorflow as tf
localpb = '/content/tflite/NAME_OF_FROZEN_GRAPH.pb'
tflite_file = '/content/NAME_OF_TFLITE_MODEL.tflite'
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
  localpb, 
  ['INPUT_NODE_NAME'], 
  ['OUTPUT_NODE_NAME'],
  input_shapes={"INPUT_NODE_NAME" : [1, 416, 416, 3]}
)
converter.experimental_new_converter = True
converter.allow_custom_ops = True

######  NO QUANTIZATION ######
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
#                                       tf.lite.OpsSet.SELECT_TF_OPS]

######  INT 8    ######
converter.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS,
                                      tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.representative_dataset = representative_dataset

tflite_model = converter.convert()
open(tflite_file,'wb').write(tflite_model)